# STEM Algorithm

## Step 1

In [18]:
import numpy as np
import pyomo.environ as pyo

#### Input

In [19]:
I, J, K = [9, 3, 2]

Ckj = [
    [-35, -40, -38],
    [20, 22, 25]
]
Aij = [
    [2, 1.75, 2.1],
    [0.5, 0.6, 0.5],
    [35, 40, 38],
    [1, 0, 0],
    [-1, 0, 0],
    [0, 1, 0],
    [0, -1, 0],
    [0, 0, 1],
    [0, 0, -1]
]
Bi = [20_000, 15_000, 450_000, 5_000, -3_000, 7_000, -4_000, 4_000, -2_000]

In [20]:
f = lambda k, *x: np.dot(x, np.array(Ckj)[k].reshape(-1, 1))

In [21]:
model = pyo.ConcreteModel()

model.i = pyo.RangeSet(0, I-1)
model.j = pyo.RangeSet(0, J-1)
model.k = pyo.RangeSet(0, K-1)

model.a = pyo.Param(model.i, model.j, initialize=dict(np.ndenumerate(Aij)))
model.b = pyo.Param(model.i, initialize=dict(enumerate(Bi)))
model.c = pyo.Param(model.k, model.j, initialize=dict(np.ndenumerate(Ckj)))

model.x = pyo.Var(model.j, domain=pyo.NonNegativeReals)


def cons_rule(model, i):
    return sum(model.a[i, j]*model.x[j] for j in model.j) <= model.b[i]
model.cons = pyo.Constraint(model.i, rule=cons_rule)

In [22]:
income_matrix = np.empty((K, K))

In [23]:
for k in range(K):
    model.obj = pyo.Objective(
        expr=sum(model.c[k, j]*model.x[j] for j in model.j),
        sense=pyo.maximize
    )

    opt = pyo.SolverFactory('cplex')
    opt.solve(model)

    x = [pyo.value(model.x[j]) for j in model.j]
    obj = pyo.value(model.obj)
    
    print(f'f_star{k} = {obj}')
    
    del model.obj

    matrix = np.array([])
    for z in range(K):
        if z == k:
            income_matrix[k, z] = obj
            continue  
        income_matrix[k, z] = f(z, *x)

f_star0 = -341000.0
f_star1 = 233200.0


In [24]:
alpha = np.empty(K)

In [25]:
for k in range(K):
    f_star = income_matrix[k, k]
    f_min = min(income_matrix[:, k])
    c = 1/np.sqrt(sum(np.array(Ckj)[k]**2))
    if c >= 1:
        c = np.floor(c)

    if income_matrix[k, k] > 0:
        alpha[k] = (f_star-f_min)*c/(f_star)
    else:
        alpha[k] = (f_min-f_star)*c/(f_min)


In [26]:
pi = alpha/sum(alpha)

In [27]:
model.pi = pyo.Param(model.k, initialize=pi, mutable=True)
model.lambd = pyo.Var(domain=pyo.NonNegativeReals)

model.obj = pyo.Objective(expr=model.lambd)
model.new_cons = pyo.ConstraintList()

model.cons_dm = pyo.ConstraintList()
for k in model.k:
    model.cons_dm.add(
        expr=model.lambd>=(income_matrix[k, k]-sum(model.c[k, j]*model.x[j] for j in model.j))*model.pi[k]
    )

In [28]:
opt = pyo.SolverFactory('cplex')
opt.solve(model)

x = [pyo.value(model.x[j]) for j in model.j]
res = [f(k, *x) for k in range(K)]

for i in range(len(res)):
    print(f'f{i} = {res[i]}')

f0 = [-368492.96778414]
f1 = [216087.47880535]


## Step 2

#### Input

In [29]:
func_old = np.array([-368492.96778414, 216087.47880535])
adjustment_data = {0: 36504.682}

In [30]:
adjustment_key = list(adjustment_data.keys())[0]
adjustment_value = adjustment_data[adjustment_key]

for k in range(K):
    model.pi[k] = 1/(I-1)
model.pi[adjustment_key] = 0



for k in range(K):
    if k == adjustment_key:
        model.new_cons.add(
            expr=sum(model.c[k, j]*model.x[j] for j in model.j)>=(func_old[k]-adjustment_value)
        )
        continue
    model.new_cons.add(
        expr=sum(model.c[k, j]*model.x[j] for j in model.j)>=func_old[k]
    )

In [31]:
opt = pyo.SolverFactory('cplex')
opt.solve(model)

x = [pyo.value(model.x[j]) for j in model.j]
res = [f(k, *x) for k in range(K)]

for i in range(len(res)):
    print(f'f{i} = {res[i]}')

model.new_cons.clear()

f0 = [-404997.64978414]
f1 = [233199.67096978]
